[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/okada-tak/deep-learning-from-scratch-2/blob/master/notebooks/ch03.ipynb)

# 3章 word2vec のまとめ
- 推論ベースの手法は、推測することを目標として、その副産物として単語の分散表現を得られる
- word2vecは推論ベースの手法であり、シンプルな2層のニューラルネットワークで構成される
- word2vecには、skip-gramモデルとCBOWモデルがある
- CBOWモデルは複数の単語（コンテキスト）からひとつの単語（ターゲット）を推測する
- skip-gramモデルは逆に、ひとつの単語（ターゲット）から複数の単語（コンテキスト）を推測する
- word2vecは重みの再学習ができるため、単語の分散表現の更新や追加が効率的に行える

## 3.1 推論ベースの手法とニューラルネットワーク
### 3.1.1 カウントベースの手法の問題点
- 語彙数が巨大
- 巨大な行列に対してSVD(特異値分解)を行うのは計算量が膨大で非現実的

### 3.1.2 推論ベースの手法の概要
- モデルはコンテキスト情報を入力として受け取り、各単語の出現する確率を出力する
- 正しい推測ができるように、コーパスを使ってモデルの学習を行う
- その学習の結果として、単語の分散表現を得られる

### 3.1.3 ニューラルネットワークにおける単語の処理方法


In [1]:
import numpy as np

c = np.array([[1,0,0,0,0,0,0]]) # 入力
W = np.random.rand(7,3) # 重み
h = np.dot(c, W) # 中間ノード：ノード数3
print(h)

[[0.27206189 0.53247467 0.32612566]]


## ■Colaboratory用
Google Colaboratoryの場合、Google Driveに  
dl-from-scratch-2/ch03  
というフォルダを用意し、そこにこのjupyter notebookを配置。  
(dl-from-scratch-3の部分は任意。)  
また、datasetフォルダとcommonフォルダを
dl-from-scratch-3/dataset  
dl-from-scratch-3/common
にコピーしておく。  

以下のセルでGoogle Driveをマウント。許可を求められるので許可する。

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## ■Colaboratory用
chdirする。

In [3]:
import sys,os
os.chdir('/content/drive/My Drive/dl-from-scratch-2/ch03')
os.getcwd()

'/content/drive/My Drive/dl-from-scratch-2/ch03'

In [4]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul

c = np.array([[1,0,0,0,0,0,0]])
W = np.random.randn(7,3)
layer = MatMul(W)
h = layer.forward(c)
print(h)

[[-0.08332784 -1.22538655 -0.49929668]]


## 3.2 シンプルなword2vec
### 3.2.1 CBOWモデルの推論処理
CBOW: continuous bag-of-words  
ターゲットは中央の単語、その周囲の単語がコンテキスト  
$\text{W}_{\text{in}}$が重みの行列であり、その各行が各単語の分散表現  



ch03/cbow_predict.py

In [5]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul


# サンプルのコンテキストデータ
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])

# 重みの初期化
W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

# レイヤの生成
in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

# 順伝搬
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)
print(s)

[[-0.4883203  -0.5887907   0.7386593  -0.28155566 -0.34191368  0.61124856
   0.01877301]]


### 3.2.2 CBOWモデルの学習
多クラス分類なので、Softmaxと交差エントロピー誤差を使う

### 3.2.3 word2vecの重みと分散表現
$\text{W}_{\text{out}}$も重みの行列であり、その各列が各単語の分散表現  
word2vecの特にskip-gramモデルでは入力側の重みだけを利用するのが多い

## 3.3 学習データの準備